# 5 Final assembly

In [10]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score
from pathlib import Path

NUM_CLASSES = 17
FOLDS = [0, 1, 2, 3, 4]

LABELS_JSON = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\data\labels\track1-qv_pipe_train.json"
SUPER_CSV    = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\data\splits\super_images_3x3_folds.csv"

BASE_RESNET   = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\framewise_resnet18"
BASE_CONVNEXT = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_convnext"
BASE_TRESNET  = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl"

# ---------- GT JSON pour tous les modèles ----------
with open(LABELS_JSON, "r") as f:
    gt_dict = json.load(f)

# ---------- CSV super-images : une ligne par vidéo avec labels_str ----------
df_super = pd.read_csv(SUPER_CSV)

# On construit un dict: video_stem (string) -> vecteur multi-hot (17)
super_label_map = {}

for _, row in df_super.iterrows():
    stem = str(row["video_stem"])  # ex: "2019"
    labels_str = str(row["labels_str"])
    if labels_str.strip() == "":
        cls = []
    else:
        cls = [int(x) for x in labels_str.split()]
    vec = np.zeros(NUM_CLASSES, dtype=int)
    for c in cls:
        vec[c] = 1
    super_label_map[stem] = vec


In [11]:
def get_array(data, keys):
    for k in keys:
        if k in data.files:
            return data[k]
    raise KeyError(f"Keys {keys} not found. Found: {data.files}")


def build_y_true_json(video_ids, gt_dict, num_classes=17):
    """Utilise le JSON track1-qv_pipe_train.json"""
    y = np.zeros((len(video_ids), num_classes), dtype=int)
    for i, vid in enumerate(video_ids):
        vid = str(vid)
        # on ramène tout à "<stem>.mp4"
        name = os.path.basename(vid)
        stem, ext = os.path.splitext(name)
        if ext != ".mp4":
            key = stem + ".mp4"
        else:
            key = stem + ext
        for c in gt_dict.get(key, []):
            y[i, c] = 1
    return y


def build_y_true_super_from_csv(raw_ids, label_map, num_classes=17):
    """
    Construit y_true pour les super-images à partir du CSV.
    raw_ids : ce qui est stocké dans le npz (chemins ou id super-image)
    On extrait le video_stem (avant le premier '_') pour matcher le CSV.
    """
    y = np.zeros((len(raw_ids), num_classes), dtype=int)
    for i, s in enumerate(raw_ids):
        s = str(s)
        name = os.path.basename(s)              # "2019_3x3.jpg" ou "2019"
        stem = os.path.splitext(name)[0]        # "2019_3x3" -> "2019_3x3"
        base = stem.split("_")[0]               # "2019"
        vec = label_map.get(base, np.zeros(num_classes, dtype=int))
        y[i] = vec
    return y


def load_preds_framewise(base_dir, model_name, fold):
    path = os.path.join(base_dir, f"preds_{model_name}_fold{fold}.npz")
    data = np.load(path, allow_pickle=True)

    video_ids = get_array(data, ["video_ids", "videos", "ids", "stems"])
    preds     = get_array(data, ["probs", "preds", "logits"])

    if preds.min() < 0 or preds.max() > 1:
        preds = 1 / (1 + np.exp(-preds))

    video_ids = np.array([str(v) for v in video_ids])
    return video_ids, preds.astype(float)


def load_preds_super(base_dir, model_name, fold):
    """
    Charge les prédictions super-images.
    On récupère les IDs bruts (paths ou noms) et on laisse build_y_true_super_from_csv
    s'occuper de retrouver le bon video_stem.
    """
    path = os.path.join(base_dir, f"preds_{model_name}_fold{fold}.npz")
    data = np.load(path, allow_pickle=True)

    raw_ids = get_array(data, ["superimage_paths", "paths", "image_paths",
                               "video_ids", "videos", "ids"])
    preds   = get_array(data, ["probs", "preds", "logits"])

    if preds.min() < 0 or preds.max() > 1:
        preds = 1 / (1 + np.exp(-preds))

    raw_ids = np.array([str(v) for v in raw_ids])
    return raw_ids, preds.astype(float)


In [12]:
def compute_ap_per_class_framewise():
    all_probs = []
    all_ytrue = []

    for fold in FOLDS:
        vids, probs = load_preds_framewise(BASE_RESNET, "resnet18", fold)
        y_true = build_y_true_json(vids, gt_dict, NUM_CLASSES)
        all_probs.append(probs)
        all_ytrue.append(y_true)

    all_probs = np.vstack(all_probs)
    all_ytrue = np.vstack(all_ytrue)
    return average_precision_score(all_ytrue, all_probs, average=None)


def compute_ap_per_class_super(base_dir, model_name):
    all_probs = []
    all_ytrue = []

    for fold in FOLDS:
        raw_ids, probs = load_preds_super(base_dir, model_name, fold)
        y_true = build_y_true_super_from_csv(raw_ids, super_label_map, NUM_CLASSES)
        all_probs.append(probs)
        all_ytrue.append(y_true)

    all_probs = np.vstack(all_probs)
    all_ytrue = np.vstack(all_ytrue)
    return average_precision_score(all_ytrue, all_probs, average=None)


print("Calcul AP par modèle...")

ap_resnet   = compute_ap_per_class_framewise()
ap_convnext = compute_ap_per_class_super(BASE_CONVNEXT, "convnext")
ap_tresnet  = compute_ap_per_class_super(BASE_TRESNET,  "tresnetxl")

print("mAP ResNet18      :", ap_resnet.mean())
print("mAP ConvNeXt      :", ap_convnext.mean())
print("mAP TResNet-XL    :", ap_tresnet.mean())


Calcul AP par modèle...
mAP ResNet18      : 0.45327347514910454
mAP ConvNeXt      : 0.07856745770430641
mAP TResNet-XL    : 0.08020312822910879


In [13]:
def compute_ap_per_class_ensemble():
    all_p_ens = []
    all_ytrue = []

    for fold in FOLDS:
        # Framewise
        vids_r, p_r = load_preds_framewise(BASE_RESNET, "resnet18", fold)
        y_r = build_y_true_json(vids_r, gt_dict, NUM_CLASSES)

        # Super-images (ConvNeXt)
        ids_c_raw, p_c = load_preds_super(BASE_CONVNEXT, "convnext", fold)
        y_c = build_y_true_super_from_csv(ids_c_raw, super_label_map, NUM_CLASSES)

        # Super-images (TResNet-XL)
        ids_t_raw, p_t = load_preds_super(BASE_TRESNET, "tresnetxl", fold)
        y_t = build_y_true_super_from_csv(ids_t_raw, super_label_map, NUM_CLASSES)

        # Canonicaliser les IDs niveau vidéo pour l'intersection
        def to_stem_list(raw_ids):
            stems = []
            for s in raw_ids:
                s = str(s)
                name = os.path.basename(s)
                stem = os.path.splitext(name)[0]
                base = stem.split("_")[0]
                stems.append(base)
            return np.array(stems)

        stems_r = to_stem_list(vids_r)       # ex: "2019"
        stems_c = to_stem_list(ids_c_raw)
        stems_t = to_stem_list(ids_t_raw)

        common = sorted(set(stems_r) & set(stems_c) & set(stems_t))
        if not common:
            continue

        idx_r = [np.where(stems_r == s)[0][0] for s in common]
        idx_c = [np.where(stems_c == s)[0][0] for s in common]
        idx_t = [np.where(stems_t == s)[0][0] for s in common]

        p_r2 = p_r[idx_r]
        p_c2 = p_c[idx_c]
        p_t2 = p_t[idx_t]

        # Les y_true sont identiques pour r/c/t, on peut en prendre un :
        y_true_fold = y_r[idx_r]

        p_ens = (p_r2 + p_c2 + p_t2) / 3.0

        all_p_ens.append(p_ens)
        all_ytrue.append(y_true_fold)

    all_p_ens = np.vstack(all_p_ens)
    all_ytrue = np.vstack(all_ytrue)

    return average_precision_score(all_ytrue, all_p_ens, average=None)


print("Calcul AP ensemble...")
ap_ensemble = compute_ap_per_class_ensemble()
print("mAP ensemble :", ap_ensemble.mean())

# ---------- DataFrame + CSV ----------
df_ap_methods = pd.DataFrame({
    "framewise_resnet18": ap_resnet,
    "superimg_convnext": ap_convnext,
    "superimg_tresnetxl": ap_tresnet,
    "ensemble_3models": ap_ensemble,
})
df_ap_methods.index = [f"class_{i:02d}" for i in range(NUM_CLASSES)]

display(df_ap_methods)

csv_path = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\ap_per_class_all_models.csv"
df_ap_methods.to_csv(csv_path, index=True)

print("\nCSV sauvé à :")
print(csv_path)


Calcul AP ensemble...
mAP ensemble : 0.46287158506412635


,framewise_resnet18,superimg_convnext,superimg_tresnetxl,ensemble_3models
class_00,0.889423,0.185527,0.185527,0.871607
class_01,0.692712,0.261564,0.266367,0.677738
class_02,0.449628,0.167796,0.161877,0.420043
class_03,0.438228,0.138448,0.142527,0.432044
class_04,0.657758,0.129178,0.133047,0.710979
class_05,0.651704,0.058168,0.059693,0.758255
class_06,0.336716,0.072133,0.075269,0.365071
class_07,0.632346,0.062320,0.065253,0.628663
class_08,0.436289,0.052633,0.059391,0.460672
class_09,0.459173,0.042457,0.044357,0.457333



CSV sauvé à :
C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\ap_per_class_all_models.csv


In [ ]:
def compute_ap_per_class_ensemble():
    all_p_ens = []
    all_ytrue = []

    for fold in FOLDS:
        # Framewise
        vids_r, p_r = load_preds_framewise(BASE_RESNET, "resnet18", fold)
        y_r = build_y_true_json(vids_r, gt_dict, NUM_CLASSES)

        # Super-images (ConvNeXt)
        ids_c_raw, p_c = load_preds_super(BASE_CONVNEXT, "convnext", fold)
        y_c = build_y_true_super_from_csv(ids_c_raw, super_label_map, NUM_CLASSES)

        # Super-images (TResNet-XL)
        ids_t_raw, p_t = load_preds_super(BASE_TRESNET, "tresnetxl", fold)
        y_t = build_y_true_super_from_csv(ids_t_raw, super_label_map, NUM_CLASSES)

        # Canonicaliser les IDs niveau vidéo pour l'intersection
        def to_stem_list(raw_ids):
            stems = []
            for s in raw_ids:
                s = str(s)
                name = os.path.basename(s)
                stem = os.path.splitext(name)[0]
                base = stem.split("_")[0]
                stems.append(base)
            return np.array(stems)

        stems_r = to_stem_list(vids_r)       # ex: "2019"
        stems_c = to_stem_list(ids_c_raw)
        stems_t = to_stem_list(ids_t_raw)

        common = sorted(set(stems_r) & set(stems_c) & set(stems_t))
        if not common:
            continue

        idx_r = [np.where(stems_r == s)[0][0] for s in common]
        idx_c = [np.where(stems_c == s)[0][0] for s in common]
        idx_t = [np.where(stems_t == s)[0][0] for s in common]

        p_r2 = p_r[idx_r]
        p_c2 = p_c[idx_c]
        p_t2 = p_t[idx_t]

        # Les y_true sont identiques pour r/c/t, on peut en prendre un :
        y_true_fold = y_r[idx_r]

        p_ens = (p_r2 + p_c2 + p_t2) / 3.0

        all_p_ens.append(p_ens)
        all_ytrue.append(y_true_fold)

    all_p_ens = np.vstack(all_p_ens)
    all_ytrue = np.vstack(all_ytrue)

    return average_precision_score(all_ytrue, all_p_ens, average=None)


print("Calcul AP ensemble...")
ap_ensemble = compute_ap_per_class_ensemble()
print("mAP ensemble :", ap_ensemble.mean())

# ---------- DataFrame + CSV ----------
df_ap_methods = pd.DataFrame({
    "framewise_resnet18": ap_resnet,
    "superimg_convnext": ap_convnext,
    "superimg_tresnetxl": ap_tresnet,
    "ensemble_3models": ap_ensemble,
})
df_ap_methods.index = [f"class_{i:02d}" for i in range(NUM_CLASSES)]

display(df_ap_methods)

csv_path = r"C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\ap_per_class_all_models.csv"
df_ap_methods.to_csv(csv_path, index=True)

print("\nCSV sauvé à :")
print(csv_path)


Calcul AP ensemble...
mAP ensemble : 0.46287158506412635


,framewise_resnet18,superimg_convnext,superimg_tresnetxl,ensemble_3models
class_00,0.889423,0.185527,0.185527,0.871607
class_01,0.692712,0.261564,0.266367,0.677738
class_02,0.449628,0.167796,0.161877,0.420043
class_03,0.438228,0.138448,0.142527,0.432044
class_04,0.657758,0.129178,0.133047,0.710979
class_05,0.651704,0.058168,0.059693,0.758255
class_06,0.336716,0.072133,0.075269,0.365071
class_07,0.632346,0.062320,0.065253,0.628663
class_08,0.436289,0.052633,0.059391,0.460672
class_09,0.459173,0.042457,0.044357,0.457333



CSV sauvé à :
C:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\ap_per_class_all_models.csv
